In [1]:
# sample number

# j = '_arcsinh'+'18.2'+'a'# '18.2'+'a'#
# j = '18'+'a'#  '_arcsinh'+'18.2'+'a'#'14.2'+'a'#'_raw'+'14.2'+'a'#
            
# '7.1a','13a','14a','15a','17a','18a','19a','20a','8.1a'

# j = '13'+'a'
# j = 'b2345'+'a'
# j = '5a'
# j = '18.2'+'a'
# j = 'b2345'+'s'#stroma - exists only in batch

j = 'b2345'+'a'
j = 'b2345'+'s'+'_prescaled'
args = {
        'j' : j,
        'feautures_ind':3,
        # 'data_folder' : 'Data_',
        'data_folder' : 'age_prescaled',
        # create visualization (umap, dbscan are always shown)
        # 'visualize':True,'plotUMAP' : True,
        
        # show figures in notebook
        'show' : True, 'saveSVG' :False,
        'recalculate_umap' : False,# calculate umap
        'recalculate_db' : False,# calculate dbscan
        'create_adjusted' : False,
        'compare' : False,# compare 11,stroma and rest of the cells
             
        }
# create_adjusted = False
# compare = False# compare 11,stroma and rest of the cells
# print(j, 'ind:',config['feautures_ind'], dir_data1, config, create_adjusted)

# parent_dir = '/home/yishai/breast_cancer_PHD_research'



use_MASTER = True





%run imports.ipynb # import all imported packages from imports.ipynb

samples = Series(df['samp'])
uniq_samples = samples.unique()# don't drop anchor samples




The directory already exists!
sample b2345s_prescaled loaded from file
j         : b2345s_prescaled
feautures_ind: 3
data_folder: age_prescaled
show      : True
saveSVG   : False
recalculate_umap: False
recalculate_db: False
create_adjusted: False
compare   : False
isSingleSample: False
isCellCycle: False
home      : /Users/yishai
dir_data  : /Users/yishai/Dropbox/CyTOF_Breast/data_yishai/age_prescaled/
dir_index : /Users/yishai/Dropbox/CyTOF_Breast/data_yishai/indexes/
dir_plots : /Users/yishai/Dropbox/CyTOF_Breast/data_yishai//plots/_sample_b2345s_prescaled_13032024_age_prescaled/
dir_functions: /Users/yishai/Desktop/breast_cancer_PHD_research/functions/
save_add  : /Users/yishai/Dropbox/CyTOF_Breast/data_yishai/
b2345s_prescaled with size: 7597
Sample is: batch
title; sampb2345s_prescaled CellCycle Based , figname; sampb2345s_prescaled_CellCycle_Based_
dict_keys(['j', 'feautures_ind', 'data_folder', 'show', 'saveSVG', 'recalculate_umap', 'recalculate_db', 'create_adjusted', 'compare

LOAD AGE DATA and order acoording to uniq samples order

In [2]:
import pathlib
home = str(pathlib.Path.home())
add = f'{home}/Dropbox/CyTOF_Breast/Kaplan_5th/'
import pandas as pd
age = pd.read_csv(add +'Age_key.txt',header=None,sep='\t')
age.columns=['sample','Age']
age['sample'] = age['sample'].str.replace('BCK-','').astype(float)  
# epinuc = pd.read_csv(add +'20240305_EPINUC_age.csv')[['sample', 'Age']]


uniq_samples = samples.unique()
uniq_samples = np.asarray([i for i in uniq_samples.astype(str) if '.1' not in i]).astype(float)# drop anchor samples
print(uniq_samples)
ages = Series([age['Age'].loc[age[age['sample']==key].index[0]] for key in uniq_samples])

print(ages)


[ 4.  5.  7.  8. 11. 13. 14. 15. 17. 18. 19. 20.]
0     61
1     87
2     37
3     46
4     76
5     73
6     64
7     63
8     62
9     67
10    44
11    69
dtype: int64


calculate mean, std matrices

In [3]:
mean = DataFrame(columns=df.columns,index = uniq_samples)
std = DataFrame(columns=df.columns,index = uniq_samples)
# std

for samp in uniq_samples:
    # print(samp)
    d = df[df['samp']==samp].copy()
    mean.loc[samp] = d.mean()
    std.loc[samp] = d.std()
    # print(d.mean())
    # print(d.std())
    # break
# samp and ind features are not needed
std.drop(['samp','ind'],axis=1,inplace=True)
mean.drop(['samp','ind'],axis=1,inplace=True)

# change index from samp number to age
std.index = ages
mean.index = ages

# std.sort_index(inplace=True)
# mean.sort_index(inplace=True)
std = std.astype(float).sort_index()
mean = mean.astype(float).sort_index()

std.head

<bound method NDFrame.head of           H4        H3      H3.3      ECad  panKeratin        K5     EpCam  H3K27me2      EZH2     gH2AX      aSMA  H3K36me2   H3K4me1   H3K9me2   H4K16ac     H2Aub  Vimentin  \
37  0.140692  0.186070  0.123217  0.419685    0.249642  0.254326  0.634569  0.299008  0.242885  0.249490  0.417868  0.363060  0.225082  0.207488  0.260721  0.235933  0.508688   
44  0.238115  0.164278  0.225352  0.512400    0.196529  0.281471  0.754269  0.351943  0.312319  0.239709  0.357393  0.448725  0.249401  0.237992  0.268553  0.298538  0.504294   
46  0.149144  0.199193  0.127994  0.507152    0.249485  0.242346  0.735499  0.403189  0.263619  0.253736  0.429046  0.450974  0.248695  0.231181  0.312811  0.340265  0.555652   
61  0.159770  0.182859  0.109841  0.408327    0.402953  0.175567  1.481203  0.249756  0.242477  0.053039  0.233463  0.211535  0.315995  0.159526  0.167498  0.086108  0.542567   
62  0.224495  0.159121  0.236354  0.517484    0.172464  0.334844  0.822544  0.35

plot with error bar

In [4]:
# import matplotlib.pyplot as plt
# import numpy as np

# def plot_with_errorBar(x,y,y_error,xlabel,ylabel,title,fname=None):
#     plt.figure(figsize=(20,10))
#     plt.title(title)
#     plt.xlabel(xlabel)
#     plt.ylabel(ylabel)
#     plt.plot(x,y)

#     # function to plot errorbar with the line plots
#     plt.errorbar(x,y,yerr=y_error,ecolor='r',elinewidth=5,fmt='s',color='k')
#     plt.grid()
#     plt.xticks(np.arange(min(x), max(x)+1, 1.0))
    
#     if fname is not None:
#         print('saving:', fname)
#         plt.savefig(fname)
#     plt.show()
    
# for col in mean.columns:
#     y = mean[col]
#     x = list(mean.index)
#     y_error = std[col]
#     xlabel = 'Age'
#     ylabel = col
#     title = f'{col} vs Age'
    
#     fname = f'{config["dir_plots"]}{col}.png'
#     # print(fname)
#     plot_with_errorBar(x,y,y_error,xlabel ,ylabel,title,fname )

correlation for each feature compared to age

In [5]:
# mean['age'] = mean.index
import seaborn as sns
mean.insert(0, 'age', mean.index  ) 
corr = pd.DataFrame(mean.copy().corr(method = 'spearman')['age'])
corr.to_csv(config['dir_plots']+config['figname']+'age_corr_spearman.csv')

pps score  - detect non liniar relations (better than correlation)

https://towardsdatascience.com/rip-correlation-introducing-the-predictive-power-score-3d90808b9598

In [6]:
import ppscore as pps
# pps.score(df, "feature_column", "target_column")
pps_score = pps.matrix(mean)


In [7]:
pps_score

pps_age = pps_score[(pps_score['x'] == 'age')+ (pps_score['y'] == 'age')]\
    .sort_values(by='ppscore',ascending=False).reset_index(drop=True)
pps_age.to_csv(config['dir_plots']+config['figname']+'age_pps_score.csv')


In [8]:
# import seaborn as sns
# import matplotlib.pyplot as plt 
# plt.figure(figsize=(20,10))
# sns.heatmap(corr,annot = True, linewidths=.5)

In [9]:
# from heatmap import Heatmap
# heatmaps = Heatmap(**config)